## 作業

In [3]:
import pandas
google = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx', index_col = 0)

In [4]:
google.head()

,username,score,comment
0,倪依芙,5,起初找不到入口，還以為沒座位，但是原來入口在旁邊樓梯～座位區不大，但是料理很好吃～肉圓的醬微...
1,Mimi龍,4,聞名已久，今天特別來現場試試，目前改成點餐機點餐， ⋯⋯更多聞名已久，今天特別來現場試試，目...
2,林昌逸,1,非常生氣服務員的態度😠服務很糟糕，麵都還沒吃完，一起身就把麵收走，擺明趕客人😡😡😡
3,林幸蓁,5,內用與外帶有不同的點餐方式內用須至店內使用機台點菜、輸入桌號並付款外帶好像只需在入口處點餐即...
4,台灣TAXI市區叫車禮車包車約拍找小鄭,5,乾淨衛生，廁所很大在地下室，有免費飲料可以喝，拉麵愛好吃，麻辣豆腐不錯、肉粽不錯，目前吃過很...


In [5]:
def convertScore(score):
  if score >= 4:
    return 'good'
  elif score <= 2:
    return 'bad'
  else: 
    return 'soso'

In [6]:
google['status'] = google['score'].map(convertScore)

In [7]:
import jieba
corpus = []
for e in google['comment'].tolist():
    corpus.append(list(jieba.cut(e)))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.940 seconds.
Prefix dict has been built successfully.


In [8]:
from gensim.models import word2vec
size = 50  # 產生多少維度 
min_count = 5 # 要算至少出現多少次數的字詞
workers = -1 # 使用多少個core 計算, -1 使用所有的core 進行計算 
window = 10  # 上下文的區間
iter = 1000 # 神經網路訓練的迭代數
sample = 1e-5 # 取樣的數量
model = word2vec.Word2Vec(corpus, 
                          workers = workers,
                          sample = sample,
                          size = size,
                          min_count=min_count,
                          window = window,
                          iter = iter)

In [10]:
#for ele in model.wv.vocab:
#  print(ele)

In [11]:
model.wv.get_vector('越南')

array([-6.2473821e-03, -1.5602022e-03,  4.3593664e-03, -7.6216026e-03,
        3.2578735e-04, -3.8040073e-03,  4.6569351e-03, -5.2936017e-03,
       -8.0661001e-03, -1.8582003e-03, -5.2584196e-03,  5.7037883e-03,
       -4.8752283e-03, -2.6064257e-03,  7.5233249e-05,  8.6057857e-03,
        7.3851845e-03,  2.5813424e-03,  8.1432397e-03, -8.6519932e-03,
       -6.9618798e-03, -4.6876217e-03, -6.9317590e-03,  4.2128176e-04,
       -8.2872005e-04, -3.7717759e-03, -7.6840674e-03, -2.0504254e-03,
        7.5913146e-03, -7.0984615e-03,  6.1308695e-03,  4.3035331e-03,
       -9.7447345e-03,  6.2935022e-03, -4.6922767e-04, -2.8430040e-03,
        2.9132112e-03,  1.7843049e-03,  5.6017037e-03,  2.7697338e-03,
       -8.8316081e-03,  7.7389916e-03, -9.2706848e-03,  7.2830929e-03,
        8.9329835e-03,  8.8444920e-03,  2.1003755e-03,  4.8506825e-04,
        7.2138994e-03,  2.1585922e-03], dtype=float32)

In [12]:
import jieba
corpus = []
tags = []
for idx,rec in google[google['status'].isin(['good', 'bad'])].iterrows():
  corpus.append(list(jieba.cut(rec.get('comment'))))
  tags.append(rec.get('status'))

In [13]:
import numpy as np
x = []
y = []
size = 50
for tag, s in zip(tags, corpus):
    vec = np.zeros(size).reshape((1, size))
    cnt = 0
    for w in s:
        if w in model:
            vec += model.wv.get_vector(w)
            cnt += 1
    if cnt > 0:
      x.append(vec / cnt)
      y.append(tag)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':


In [15]:
x = np.concatenate(x, axis = 0)

In [16]:
x.shape

(440, 50)

In [17]:
len(y)

440

In [19]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [20]:
train_x.shape

(352, 50)

In [21]:
test_x.shape

(88, 50)

In [23]:
len(train_y)

352

In [24]:
len(test_y)

88

In [29]:
from sklearn.svm import SVC
clf = SVC(kernel = 'rbf')
clf.fit(train_x, train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [30]:
pred_y = clf.predict(test_x)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.7045454545454546

In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, pred_y)

array([[29, 11],
       [15, 33]])

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(train_x, train_y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [35]:
pred_y = clf.predict(test_x)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.625

## Selenium
- https://chromedriver.chromium.org/
- https://www.largitdata.com/course_list/15

In [37]:
! pip install selenium

     |████████████████████████████████| 911kB 4.5MB/s 


In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('/opt/local/common/chromedriver')
driver.get('https://www.google.com/search?rlz=1C5CHFA_enTW718TW718&ei=gYZPX4XfFMO4mAWk3I6gDQ&q=google+%E7%8E%89%E6%AC%A3%E7%8F%8D&oq=google+%E7%8E%89%E6%AC%A3%E7%8F%8D&gs_lcp=CgZwc3ktYWIQAzIFCCEQoAEyBQghEKABOgQIABAeOgYIABAIEB5Q5xhYv1dg5l1oEXAAeAGAAcMBiAHFDZIBBDE4LjOYAQCgAQGqAQdnd3Mtd2l6wAEB&sclient=psy-ab&ved=0ahUKEwjFo4yxs8rrAhVDHKYKHSSuA9QQ4dUDCA0&uact=5')

In [ ]:
driver.find_element_by_css_selector('a[data-async-trigger="reviewDialog"]').click()

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source)

In [ ]:
for review in soup.select('.review-snippet'):
    print(review.text)

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

## BERT

In [1]:
! pip install transformers

     |████████████████████████████████| 890kB 8.0MB/s 
     |████████████████████████████████| 3.0MB 29.5MB/s 
     |████████████████████████████████| 1.1MB 52.7MB/s 
     |████████████████████████████████| 890kB 41.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=94a65c8233aaba59493b5b3bfe37c80f9208d9ebc7e98daa3dd14d89c38abee3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

## Simple Transformers

In [ ]:
! pip install simpletransformers

In [1]:
import pandas as pd
df = pd.read_excel('https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/yahoo_movie.xlsx', index_col = 0)

In [2]:
df.head()

,content,stars,title,status
0,"不知道耶! 看完整個無感\r\n有種覺得就是女兒""給蕭""害死了老爸\r\n然後...既然可以...",3,古墓奇兵,soso
1,很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。女主角跟爸爸...,5,古墓奇兵,good
2,說這個不好看的話，那我還真不知道，還有什麼片是您可以去看得了。電影好看，但話說羅拉有裝可以撿...,5,古墓奇兵,good
3,父女重逢真的很讓人感動，五顆星。,5,古墓奇兵,good
4,劇情雖然老套，但仍拍出新意，古墓能殺人的方式不就是機關和毒，要求亂七八糟的觀眾，你看喪尸片看多了。,4,古墓奇兵,good


In [3]:
df2 = df[df['status'].isin(['good', 'bad'])]

In [4]:
df2 = df2[['content', 'status']]

In [5]:
df2.head()

,content,status
1,很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。女主角跟爸爸...,good
2,說這個不好看的話，那我還真不知道，還有什麼片是您可以去看得了。電影好看，但話說羅拉有裝可以撿...,good
3,父女重逢真的很讓人感動，五顆星。,good
4,劇情雖然老套，但仍拍出新意，古墓能殺人的方式不就是機關和毒，要求亂七八糟的觀眾，你看喪尸片看多了。,good
5,每個演員都長得像路人，劇情解謎上都沒有作一些鋪陳，以致於每個謎題只有主角轉一轉就都破解了，覺...,bad


In [6]:
df2['status'] = df2['status'].replace({'good':1, 'bad': 0})

In [7]:
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df2, test_size = 0.33, random_state = 42)

In [8]:
train_df.head()

,content,status
110,成功創造新角色，也成功的揮別了舊角色的包袱，但卻又呼應了舊角色的命運，成功的續集電影,1
56,我從1977年星際大戰四部曲上映以來，每部都是初上映就到電影院觀賞。但「最後的絕地武士 」這...,0
56,太令人失望...浪費我的金錢和時間,0
71,如提,0
343,劇情拼湊 失望的一集 但喜歡看大場面的還是進電影院吧,0


In [9]:
train_df.shape

(656, 2)

In [11]:
eval_df.head()

,content,status
235,劇情爛⋯到⋯不⋯行⋯編劇要出來解釋一下嗎⋯,0
117,"如題,不只是超級英雄而已！！",1
73,比上一集難看。。。找一個女胖子演這個角色沒人了是嗎？,0
129,給兩顆星是因為上一集更爛,0
79,原本只想打發時間，沒想到蠻好看的,1


In [12]:
eval_df.shape

(324, 2)

### Hugging Face Models
- https://huggingface.co/models

In [17]:
from simpletransformers.classification import ClassificationModel
#?ClassificationModel
model = ClassificationModel('bert', 'bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
result, model_output, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:760: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [19]:
result

{'eval_loss': 0.8044313512197355,
 'fn': 196,
 'fp': 0,
 'mcc': 0.0,
 'tn': 128,
 'tp': 0}

In [ ]:
# Confusion Matrix
# 0, 0
# 196 128

In [21]:
! nvidia-smi

Wed Sep  2 13:02:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    38W / 300W |   3867MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
model.train_model(train_df, args = {'overwrite_output_dir': True})

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:288: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [22]:
result, model_output, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:760: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [23]:
result

{'eval_loss': 0.3834630282732045,
 'fn': 33,
 'fp': 15,
 'mcc': 0.701965043577896,
 'tn': 113,
 'tp': 163}

In [24]:
# Confusion Matrix
#163 15
#33  113

In [25]:
(163 + 113) / (163 + 113 + 15 + 33)

0.8518518518518519

In [26]:
model.predict(['這是一部絕世好片...不懂為什麼這麼多人看不懂... 其實很好理解吧..一開始我就猜到導演想做什麼了... 能拍出這種片子...電影真的很讓人感動~~~~'])

(array([1]), array([[-1.7522002,  1.3256476]], dtype=float32))

In [27]:

model.predict(['智商低於150的，不懂什麼是逆轉，也不是諾藍愛好者的，真的不要看。 小弟是認為對話太跳，畫面太跳，閃神一下，就會看不懂，只好陷入昏迷了。'])

(array([0]), array([[ 1.2335975 , -0.92744744]], dtype=float32))

In [ ]:
# mcc == 1
100 0
0  100

# mcc == 0
50 50
50 50

0 50
50 0

## BERT WWM
- https://arxiv.org/pdf/1906.08101.pdf

In [28]:
from simpletransformers.classification import ClassificationModel
#?ClassificationModel
model = ClassificationModel('bert', 'hfl/chinese-bert-wwm')

Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint

In [29]:
result, model_output, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:760: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [30]:
result

{'eval_loss': 0.7067622338853231,
 'fn': 56,
 'fp': 114,
 'mcc': -0.2094672926298156,
 'tn': 14,
 'tp': 140}

In [31]:
model.train_model(train_df, args = {'overwrite_output_dir': True})

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:288: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [32]:
result, model_output, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:760: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [33]:
result

{'eval_loss': 0.44095945358276367,
 'fn': 46,
 'fp': 18,
 'mcc': 0.611187498553047,
 'tn': 110,
 'tp': 150}

In [34]:
(150 + 110) / (150 + 110 + 46 + 18)

0.8024691358024691

## OpenCC

In [35]:
! pip install opencc

     |████████████████████████████████| 1.3MB 9.5MB/s 


In [36]:
import opencc
converter = opencc.OpenCC('s2t.json')
converter.convert('汉字') 

'漢字'

In [37]:
import opencc
converter = opencc.OpenCC('t2s.json')
converter.convert('漢字') 

'汉字'

In [39]:
import opencc
converter = opencc.OpenCC('t2s.json')
train_df['content'] = train_df['content'].map(lambda e: converter.convert(e))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
eval_df['content'] = eval_df['content'].map(lambda e: converter.convert(e))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
train_df.head()

,content,status
110,成功创造新角色，也成功的挥别了旧角色的包袱，但却又呼应了旧角色的命运，成功的续集电影,1
56,我从1977年星际大战四部曲上映以来，每部都是初上映就到电影院观赏。但「最后的绝地武士 」这...,0
56,太令人失望...浪费我的金钱和时间,0
71,如提,0
343,剧情拼凑 失望的一集 但喜欢看大场面的还是进电影院吧,0


In [42]:
model = ClassificationModel('bert', 'hfl/chinese-bert-wwm')

Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint

In [43]:
model.train_model(train_df, args = {'overwrite_output_dir': True})

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:288: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [44]:
result, model_output, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:760: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [45]:
result

{'eval_loss': 0.34890466783104873,
 'fn': 30,
 'fp': 18,
 'mcc': 0.6970401070193235,
 'tn': 110,
 'tp': 166}

In [46]:
(166 + 110) / (166 + 110 + 30 + 18)

0.8518518518518519